In [115]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

## function to load images and labels

In [116]:
def load_classes(classes_path):
    with open(classes_path, 'r') as file:
        classes = file.read().splitlines()
    return classes

def load_dataset(dataset_path, classes):
    images = []
    labels = []
    for folder_name in ['train', 'test']:  # removed 'validation'
        folder_path = os.path.join(dataset_path, folder_name)
        annotations_path = os.path.join(folder_path, '_annotations.txt')
        with open(annotations_path, 'r') as file:
            for line in file.readlines():
                parts = line.strip().split()
                image_path = os.path.join(folder_path, parts[0])
                bounding_boxes = parts[1:]
                image = load_img(image_path, target_size=(64, 64))
                image = img_to_array(image)
                images.append(image)
                labels_for_image = []
                for bbox in bounding_boxes:
                    x_min, y_min, x_max, y_max, class_id = map(int, bbox.split(','))
                    class_label = classes[class_id]  # convert class ID to class label
                    labels_for_image.append([x_min, y_min, x_max, y_max, class_label])
                labels.append(labels_for_image)
    images = np.array(images)
    return images, labels  # labels is a list of lists of bounding boxes

## function to preprocess images and labels

In [117]:
def preprocess(images, labels):
    images = images.astype('float32') / 255.0
    labels = to_categorical(labels)
    return images, labels

## define model 

In [118]:
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

## Load and preprocess the dataset

In [119]:
classes_path = r'C:\Users\suraj\OneDrive\Desktop\Projects\internship_applications_project\website_snapshot_element_identifier\Website Screenshots.v1-raw.yolov4pytorch\train\_classes.txt'
classes = load_classes(classes_path)
dataset_path = r'C:\Users\suraj\OneDrive\Desktop\Projects\internship_applications_project\website_snapshot_element_identifier\Website Screenshots.v1-raw.yolov4pytorch'
images, labels = load_dataset(dataset_path, classes)

## train test split

In [120]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

## create model

In [121]:
input_shape = X_train.shape[1:]
num_classes = len(set(class_label for sublist in y_train for _, _, _, _, class_label in sublist))
print(num_classes)

8


## compile model

In [122]:
from keras.models import Sequential
from keras.layers import Dense

input_dim = X_train.shape[1] * X_train.shape[2] * X_train.shape[3]  # calculate input_dim

# Flatten the images
X_train = X_train.reshape(X_train.shape[0], input_dim)
X_val = X_val.reshape(X_val.shape[0], input_dim)

model = Sequential()
model.add(Dense(32, input_shape=(input_dim,), activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## train model

In [123]:
#If your y_train and y_val are lists of lists where the last element is the class label, you can convert them to NumPy arrays of class labels 
import numpy as np

y_train = np.array([label[-1] for label in y_train])
y_val = np.array([label[-1] for label in y_val])

In [124]:
from sklearn.preprocessing import LabelEncoder
y_train_labels = y_train[:, -1]
y_val_labels = y_val[:, -1]

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train_labels)
y_val_encoded = encoder.transform(y_val_labels)

y_train = to_categorical(y_train_encoded, num_classes=8)
y_val = to_categorical(y_val_encoded, num_classes=8)

In [125]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
49/49 [==============================] - 1s 9ms/step - loss: 442.3643 - accuracy: 0.1574 - val_loss: 2.0694 - val_accuracy: 0.2021
Epoch 2/10
49/49 [==============================] - 0s 6ms/step - loss: 2.0611 - accuracy: 0.1788 - val_loss: 2.0510 - val_accuracy: 0.2021
Epoch 3/10
49/49 [==============================] - 0s 7ms/step - loss: 2.0421 - accuracy: 0.2513 - val_loss: 2.0321 - val_accuracy: 0.2824
Epoch 4/10
49/49 [==============================] - 0s 7ms/step - loss: 2.0232 - accuracy: 0.3089 - val_loss: 2.0135 - val_accuracy: 0.2824
Epoch 5/10
49/49 [==============================] - 0s 7ms/step - loss: 2.0043 - accuracy: 0.3089 - val_loss: 1.9949 - val_accuracy: 0.2824
Epoch 6/10
49/49 [==============================] - 0s 8ms/step - loss: 1.9862 - accuracy: 0.3089 - val_loss: 1.9778 - val_accuracy: 0.2824
Epoch 7/10
49/49 [==============================] - 0s 7ms/step - loss: 1.9691 - accuracy: 0.3089 - val_loss: 1.9611 - val_accuracy: 0.2824
Epoch 8/10
49/49 [

## model evaluation

In [126]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Loss:', loss)
print('Accuracy:', accuracy)

13/13 [==============================] - 0s 1ms/step - loss: 1.9166 - accuracy: 0.2824
Loss: 1.9165962934494019
Accuracy: 0.28238341212272644


In [140]:
from PIL import Image
import numpy as np

# Load the image
image = Image.open(r'C:\Users\suraj\OneDrive\Desktop\Projects\internship_applications_project\website_snapshot_element_identifier\Website Screenshots.v1-raw.yolov4pytorch\test\addons_mozilla_org_png.rf.8pWFwr9ZblJ92BQghXBW.jpg')


# Resize the image to match the input shape of your model
# Your model seems to have been trained on images of size 64x64
image = image.resize((64, 64))

# Convert the image to a numpy array and normalize the pixel values (if you did so during training)
image_array = np.array(image) / 255.0

# Add an extra dimension to match the model's input shape (model expects a batch)
image_array = np.expand_dims(image_array, axis=0)

# Flatten the image
image_array = image_array.reshape((1, -1))

# Use the model to predict the class of the image
predictions = model.predict(image_array)

# The predictions are probabilities for each class, so we'll take the class with the highest probability
predicted_class = np.argmax(predictions, axis=-1)

print(f'Predicted class: {predicted_class}')
print(predictions[0])

1/1 [==============================] - 0s 14ms/step
Predicted class: [0]
[0.17586988 0.0982605  0.1093415  0.1028178  0.1540752  0.08210292
 0.11669035 0.16084185]
